In [12]:
from simulate_module import *

In [13]:
import pandas as pd
from pathlib import Path
import numpy as np
np.random.seed(1234)

In [14]:
sigma_setting = {"high_bw": [10, .2],
                "medium_bw": [1, .2],
                "low_bw": [.5, .2]}

In [15]:
def basis(x, knots=None, degree = 10):
    if knots is None:
        knots = np.linspace(0, 1, 10)
        
    H = [np.ones(len(x))]
    for k in range(len(knots)):
        for j in range(1, degree + 1):
            H.append(pos(x - knots[k]) ** j)
    
    return np.vstack(H).T
def random_functions(n_tasks, k_clusters, sigma_between, sigma_within):
    np.random.seed(1234)
    betas, zs = gaussian_mixture(n_tasks, k_clusters, sigma_between, sigma_within)
    functions = []
    for beta in betas:
        functions.append(spline_fun(beta))
        
    return functions, betas, zs

def gaussian_mixture(n_samples, k_clusters, sigma_between, sigma_within, dim = 101):
    means = np.random.normal(0, sigma_between, (k_clusters, dim))
    cluster_props = np.random.dirichlet(k_clusters * [1.5])
    
    betas, zs = [], []
    for task in range(n_samples):
        z = np.random.choice(range(k_clusters), p = cluster_props)
        delta = np.random.normal(0, sigma_within, (1, dim))
        betas.append(means[z] + delta)
        zs.append(z)
        
    return np.vstack(betas), zs
        
def spline_fun(beta):
    def f(x):
        return np.matmul(basis(x), beta)
    return f
    




In [15]:
base_dir = Path("debug_minmax")
base_dir.mkdir(exist_ok = True)
metadata = []
i = 0
for conservative in [True]:
    for target_test_size in [.4]:#np.linspace(0.2,0.8,4):
        for s in sigma_setting:
            for target_task in [0]:
                for decay_rate in [0.5]:
                    for model_type in ["nn"]:
                        metadata.append({
                            "path": "exp" + str(i),
                            "n_tasks": 15,
                            "conservative": conservative,
                            "target_test_size": target_test_size,
                            "model_type": model_type,
                            "s": s,
                            "target_task": target_task,
                            "decay_rate": decay_rate
                        })
                        i += 1

metadf = pd.concat([pd.DataFrame(m, index=[i]) for i, m in enumerate(metadata)])
metadf.to_csv( base_dir / "metadata.csv")

sigma_setting = {"high_bw": [10, .2],
                "medium_bw": [1, .2],
                "low_bw": [.5, .2]}




In [7]:
for i, args in enumerate(metadata):
    working_path = Path(base_dir / args["path"])
    working_path.mkdir(parents = True, exist_ok = True)
    
    if args["model_type"] == "lm":
        loss_fn = mse
    elif args["model_type"] == "nn":
        loss_fn =  torch.nn.MSELoss()
    
    
    # generate data ------------------------------------------------
    np.random.seed(1234)
    f, betas, zs = random_functions(args["n_tasks"], 6,
                                    sigma_between = sigma_setting[args["s"]][0],
                                    sigma_within = sigma_setting[args["s"]][-1])
    result = []
    for i, fi in enumerate(f):
        x = np.random.uniform(0, 1, 100)
        result.append({
            "task": i,
            "x": x,
            "f": fi(x),
            "y": fi(x) + np.random.normal(0, .1, len(x))
        })
    # save data
    data_df = pd.concat([pd.DataFrame(r) for r in result])
    data_df.to_csv(working_path.joinpath("tasks.csv"), index = False)
    data_df = data_df.reset_index()
    betas_df = np.hstack([np.arange(args["n_tasks"])[:, np.newaxis], np.array(zs)[:, np.newaxis], betas])
    betas_df = pd.DataFrame(betas_df)
    betas_df.columns = ["task", "cluster"] + [f"beta{i}" for i in range(betas.shape[1])]
    betas_df.to_csv(working_path / "betas.csv", index = False)
    data_dict = data_df.to_dict(orient = "list")
    
    input_data = prepare_input(data_dict,
                               target_task = args["target_task"],
                                target_test_size = args["target_test_size"],
                                preprocess = True)
    pd.DataFrame.from_dict(input_data["data_dict"]).to_csv(working_path / "tasks_processed.csv",
                                                               index = False)
        # bandit selection
    losses, alpha, beta, bandit_selects, pi, bl = bandit_source_train(input_data = input_data,
                                                                          model_class = args["model_type"],
                                                                          batch_size = 8,
                                                                          decay_rate = args["decay_rate"],
                                                                          n_it = 100,
                                                                          loss_fn =  loss_fn,
                                                                          conservative = args["conservative"],
                                                                     save_path = working_path)
        # save outputs
    save_files(working_path, alpha, beta, losses, bandit_selects, pi, bl)
    print(args)
    print(str(bl["bandit_final"]), str(bl["target_train"]))

{'path': 'exp0', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'high_bw', 'target_task': 0, 'decay_rate': 0.5}
[0.08234824240207672] [0.08745700865983963]
{'path': 'exp1', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'medium_bw', 'target_task': 0, 'decay_rate': 0.5}
[0.08102180063724518] [0.10859353840351105]
{'path': 'exp2', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'low_bw', 'target_task': 0, 'decay_rate': 0.5}
[0.07089298218488693] [0.06820078939199448]


In [8]:
working_path = Path("debug_yaxis")
working_path.mkdir(exist_ok = True)
np.random.seed(123)
f, betas, zs = random_functions(15, 6,
                                sigma_between = 10,
                                sigma_within = .2)
result = []
for i, fi in enumerate(f):
    x = np.random.uniform(0, 1, 100)
    result.append({
        "task": i,
        "x": x,
        "f": fi(x),
        "y": fi(x) + np.random.normal(0, .1, len(x))
    })
# save data
data_df = pd.concat([pd.DataFrame(r) for r in result])
data_df = data_df.reset_index()
data_dict = data_df.to_dict(orient = "list")
data_df.to_csv(working_path / "tasks.csv", index = False)
betas_df = np.hstack([np.arange(15)[:, np.newaxis], np.array(zs)[:, np.newaxis], betas])
betas_df = pd.DataFrame(betas_df)
betas_df.columns = ["task", "cluster"] + [f"beta{i}" for i in range(betas.shape[1])]
betas_df.to_csv(working_path / "betas.csv", index = False)
    



In [19]:
input_data = prepare_input(data_dict,
                                   target_task = 10,
                                   target_test_size = .4,
                                  preprocess = True)

In [20]:
model = nn(n_inputs = 1)
n_it = 100
batch_size = 8
decay_rate = .5
conservative = True
loss_fn = torch.nn.MSELoss()
lr = 5e-4
n_epochs = 10



In [21]:
mod = nn()
val_x, val_y = mod.prepare_data(input_data["X_target_val"], input_data["y_target_val"])
X_current, y_current = mod.prepare_data(input_data["X_target_train"], input_data["y_target_train"])
mod.fit( val_x, val_y, n_epochs = 50, lr = lr)

mod.save(path = working_path.joinpath("val_c"), x_new = X_current, y_new = y_current, para = True)
mod.save(path = working_path.joinpath("val"), x_new = val_x, y_new = val_y, para = True)

In [62]:
bandit_selects = [None]
# initialize hyperparameters
alpha = dict.fromkeys(input_data["source_task"], [1])
beta = dict.fromkeys(input_data["source_task"], [1])
pi = dict.fromkeys(input_data["source_task"], [0])

# initialize model from target training data
mod = nn()
val_x, val_y = mod.prepare_data(input_data["X_target_val"], input_data["y_target_val"])
X_current, y_current = mod.prepare_data(input_data["X_target_train"], input_data["y_target_train"])
mod.fit( X_current, y_current, n_epochs = n_epochs, lr = lr)

l = mod.evaluate(val_x, val_y)
losses = [l]

model_old = copy.deepcopy(mod.model) # *********************

for t in range(n_it):
    
    # select bandit
    bandit_current, pi = get_bandit(input_data, alpha, beta,t, pi)
    bandit_selects.append(bandit_current)
    
    # set training data at this iteration
    X_current, y_current, _ = subset_data(input_data["source_dict"], 
                               key_value = bandit_current,
                               key_name = "task", test_size = 0)
    batch_id = random.choices(list(range(0, len(y_current))), k = batch_size)
    X_current, y_current = X_current[batch_id, :], y_current[batch_id]
    X_current = np.concatenate((X_current, input_data["X_target_val"]), axis = 0)
    y_current = np.concatenate((y_current, input_data["y_target_val"]), axis = 0)
    X_current, y_current = mod.prepare_data(X_current, y_current)
    
    # train model
    mod.initialize()
    #mod = nn()
    mod.fit(X_current, y_current, loss_fn = loss_fn, n_epochs = n_epochs, lr = lr)
    
    
    
    # combine parameters with previous model
    mod.combine_with_old(model_old, decay_rate = .9)
    #print(t, ", current = ", mod.model[4].weight[0,0].detach().numpy(),
         # ", old = ", model_old[4].weight[0,0].detach().numpy())
    
    model_old = copy.deepcopy(mod.model)
    
    
    # evaluate model
    l = mod.evaluate(val_x, val_y, loss_fn = loss_fn)
    losses += [l]
    
    
    model_old = copy.deepcopy(mod.model)
    
    # update bandit parameters
    if conservative:
        thres = 100000
    else:
        thres = avg_loss(bandit_selects, losses, bandit_current)
    alpha, beta = update_hyper_para(alpha, beta, t, losses,
                                    bandit_current,
                                    thres = thres
                                   )
    
    mod.save(path = working_path.joinpath("current" + str(t)), x_new = X_current, y_new = y_current, para = True)
    mod.save(path = working_path.joinpath(str(t)), x_new = val_x, y_new = val_y, para = True)